# BIM6065C-E22

Assignment designed by Albert Feghaly

Answers wriiten by Alya Zeinaty

## Summary

_Goal of the assignment: determining with a certain degree of confidence how likely it is that a random drawing of six 6-sided dice gives a sum over 50._

<br>
A. Trying random resamplings <br> <br>
B. Bootstrap approach to test our hypothesis <br> <br>
        -- Calculating p-values and confidence intervals <br>
        -- Concluding <br>

#### 0. Setup your environment

In [1]:
from random import choices, seed
import altair as alt
import pandas as pd
import os
import numpy as np

def json_dir(data, data_dir='altairdata'):
    os.makedirs(data_dir, exist_ok=True)
    return alt.pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}'))
alt.data_transformers.register('json_dir', json_dir)
alt.data_transformers.enable('json_dir')

DataTransformerRegistry.enable('json_dir')

#### 1. Statistics
Here we review code used during the presentation and break it down

## A. Random resamplings

_Code from presenation_

In [2]:
from random import sample, seed
values = [1,2,3,4,5,6]
seed(0)
print(sum([sum(choices(values, k=10)) >= 50 for _ in range(100000)])/100000)

0.00302


_can be re-written as_

##### i) Define your data and parameters

In [3]:
values = [1,2,3,4,5,6]  # possible outcomes from a 6-sided dice

n = 100000  # number of random samplings

##### ii) Calculate p-value by performing random samplings

In [4]:
rs = []  # define list for storing the sums

seed(0)  # use just before calling a function from 'random' to get reproducible results

for _ in range(n):
    random_values = choices(values, k=10)  # get 10 values from 'all_values' without replacement
    random_sum = sum(random_values)
    rs.append(random_sum)  # append means add the value 'random_sum' to rs
                           # another way to write it is: rs = rs + [random_sum]

at_least_50 = []
for one_rs in rs:
    at_least_50.append(one_rs >= 50)

n_true = sum(at_least_50)
pval = n_true/n
print(pval)

# Some notes
#     Using the variable '_' tells python to ignore the value assigned to it.
#     Setting the seed to 0 is arbitrary, any number would do, as long as you always the same number.
#     When adding a value to an existing list, you do so with 'append'.

0.00302


##### ii) Visualize your simulated data using altair

In [5]:
data = pd.DataFrame({'RandomValues': rs})

bars = alt.Chart(data).mark_bar().encode(
    x=alt.X("RandomValues:Q", bin=alt.Bin(extent=[10, 60], step=1), title="RandomValues"),
    y="count()"
)

line_df = pd.DataFrame({'a': [50]})

vertline = alt.Chart(line_df).mark_rule(color='red').encode(
    x=alt.X('a:Q', title='')
)

chart = alt.layer(
    bars, vertline
).properties(
    height=500,
    width=400
)

chart.interactive()  # you can play around with it in interactive mode

C:\Users\Alya\anaconda3\lib\site-packages\altair\utils\data.py:226: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  warnings.warn(


alt.LayerChart(...)

## B. Bootstrap approach

The bootstrap approach is a statistical method that works in many, many scenarios. In general, if a parametric (t-test, ANOVA, etc.) method is not applicable, you want to consider bootstrapping. Generally, you need a minimum number of observations for the test to make sense (at least 5 or 6). The principle behing this technique is to _imitate_ replicates of your data, by recreating different combinations of it, which is achieved by resampling **with** replacement.

_Code from presentation_

In [6]:
from random import choices, seed
v1 = [3.47, 3.51, 3.54, 3.58, 3.62, 3.64, 3.65, 3.68, 3.71] ; lt = len(v1)
v2 = [3.41, 3.42, 3.44, 3.45, 3.46, 3.51, 3.54, 3.63, 3.64] ; n = 100000 ; alpha = 0.01
diff = sum(v1)/lt - sum(v2)/lt ; print(round(diff, 5))
seed(0) ; stds = sorted([sum(choices(v1, k=lt))/lt - sum(choices(v2, k=lt))/lt for _ in range(n)])
print('[', round(stds[int(n*alpha/2)], 5), ',', round(stds[n - int(n*alpha/2)], 5), ']')

0.1
[ 0.00111 , 0.19 ]


_can be re-written as_

##### i) Define your data and parameters

In [7]:
v1 = [3.47, 3.51, 3.54, 3.58, 3.62, 3.64, 3.65, 3.68, 3.71]  # list of values #1
v2 = [3.41, 3.42, 3.44, 3.45, 3.46, 3.51, 3.54, 3.63, 3.64]  # list of values #2


n = 100000  # number of bootstraps to compute
alpha = 0.01  # alpha level

##### ii) [Optional] Visualize your data using altair

In [8]:
# Put v_1 and v_2 in a dataset
values = v1 + v2
origin = ['v1']*len(v1) + ['v2']*len(v2)

data = pd.DataFrame([values, origin], index=['value', 'origin'])
data = data.transpose()

# Plot your dataset using altair
alt.Chart(data).mark_point(filled=True).encode(
    x=alt.X("value", scale=alt.Scale(domain=[3, 4])),
    y=alt.Y('origin'),
    color=alt.Color('origin')
).interactive()

alt.Chart(...)

##### iii) Construct bootstrap CI

We will calculate the confidence interval around the difference in means of the 2 dice sets

In [9]:
mean_1 = sum(v1) / len(v1)  # mean of values #1
mean_2 = sum(v2) / len(v2)  # mean of values #2

diff = mean_1 - mean_2  # difference of the means

print('Difference = %.5g' % diff)  # print difference rounding up to 5 decimals

stds = []  # empty list where the standard deviations from the mean will be stored
seed(0)  # set your random state in order to get the same result from your random samplings
for _ in range(n):  # iterate from 0 to 999,999 (100,000 iterations)
    random_boot_1 = choices(v1, k=len(v1))  # resample v1 with replacement
    random_boot_2 = choices(v2, k=len(v2))  # resample v2 with replacement
    boot_mean_1 = sum(random_boot_1) / len(v1)
    boot_mean_2 = sum(random_boot_2) / len(v2)
    new_diff = boot_mean_1 - boot_mean_2
    stds.append(new_diff)  # add your calculated difference to the list with the 'append' method

stds_sorted = sorted(stds)  # sort your values from smallest to largest

quantile_at_0_5 = int(n*(alpha/2))  # get the position of your 0.5% quantile in stds_sorted (as an integer)
quantile_at_99_5 = n - int(n*(alpha/2))  # get the position of your 99.5% quantile in stds_sorted (as an integer)

print('0.5%% quantile position = %d // 99.5%% quantile_position = %d' % (quantile_at_0_5, quantile_at_99_5))

q0_5 = stds_sorted[quantile_at_0_5]  # 0.5% quantile value from stds_sorted
q99_5 = stds_sorted[quantile_at_99_5]  # 99.5% quantile value from stds_sorted

contains_0 = (q0_5 < 0) and (q99_5 > 0)  # True if 0 inside CI, False if not

print('CI = [%.5g , %.5g]' % (q0_5, q99_5))
print('Is 0 outside the confidence interval ?', not contains_0)

Difference = 0.1
0.5% quantile position = 500 // 99.5% quantile_position = 99500
CI = [0.0011111 , 0.19]
Is 0 outside the confidence interval ? True


##### iv) Visualize your simulated data using altair

In [10]:
data = pd.DataFrame({'RandomValues': stds})
temp = pd.DataFrame({'a':[stds_sorted[int(n*alpha/2)]], 'b': [stds_sorted[n-int(n*alpha/2)]]})

bars = alt.Chart(data).mark_bar().encode(
    alt.X("RandomValues:Q", bin=alt.Bin(extent=[-0.1, 0.3], step=0.01), title='RandomValues'),
    y='count()',
)

vertline1 = alt.Chart(temp).mark_rule(color='red').encode(
    x=alt.X('a:Q', title='')
)

vertline2 = alt.Chart(temp).mark_rule(color='red').encode(
    x=alt.X('b:Q', title='')
)

chart = alt.layer(
    bars, vertline1, vertline2
).properties(
    height=500,
    width=400
)

chart.interactive()

alt.LayerChart(...)

##### iv) Let's now calculate the p-value equivalent of the CI

We will calculate the two-tailed p-value for the alternative hypothesis that the 2 means are not equal

_Let's break it down so that you can add your comments_

In [11]:
n = 100000  # number of bootstraps to compute
alpha = 0.01  # alpha level

##### describe what n is and why we chose this number, and describe what alpha is and why is it defined at the top

In [12]:
mean_1 = sum(v1) / len(v1)  # mean of values #1
mean_2 = sum(v2) / len(v2)  # mean of values #2

overall_mean = sum(v1 + v2) / (len(v1) + len(v2))

##### what are we calculating here?

In [13]:
mean_1

3.5999999999999996

In [14]:
mean_2

3.4999999999999996

In [15]:
mean_1-mean_2

0.10000000000000009

In [16]:
centered_v1 = []
for x in v1:
    centered_x = x - mean_1 + overall_mean
    centered_x = round(centered_x, 5)  # this line is optional, used just to make the numbers pretty
    centered_v1.append(centered_x)
    
centered_v2 = []
for x in v2:
    centered_x = x - mean_2 + overall_mean
    centered_x = round(centered_x, 5)  # this line is optional, used just to make the numbers pretty
    centered_v2.append(centered_x)

##### what is this calculation and why are we doing it?

In [17]:
### Find the 2-tailed 99% confidence interval of the difference of the means, remembering that
### the absolute difference of the means was found to be mean_2-mean_1=0.1
from statistics import mean
seed(0) 
lt=len(centered_v1)
stds = sorted([sum(choices(centered_v1, k=lt))/lt - sum(choices(centered_v2, k=lt))/lt for _ in range(n)])
print('The confidence interval offset by 0.1 is','[', round(stds[int(n*alpha/2)]+0.1, 5), ',', round(stds[n - int(n*alpha/2)]+0.1, 5), ']')#intervalle de confiance centré sur la moyenne overall
print('The confidence interval offset by 0 is','[', round(stds[int(n*alpha/2)], 5), ',', round(stds[n - int(n*alpha/2)], 5), ']')
print('Remark: To get the CI for the null hypothesis, we could have simply printed out the CI of the uncentered data offset by -0.1 .')

The confidence interval offset by 0.1 is [ 0.00111 , 0.19 ]
The confidence interval offset by 0 is [ -0.09889 , 0.09 ]
Remark: To get the CI for the null hypothesis, we could have simply printed out the CI of the uncentered data offset by -0.1 .


In [18]:
### Calculate the number of times we get a result at least as extreme as our difference of the means
extreme=len([i for i in stds if i>=0.1 or i<=-0.1])
print('Since we are looking at a two-tailed CI, we need to examines values at least as extreme as the values observed at both sides of the interval, which means higher than 0.1 or lower than -0.1; however, it does not make a lot of sense because we can expect that the genie will never bias the data in his disfavor.')
print('The number of times we get a result at least as extreme as our difference of the means is ', extreme, '.')    

Since we are looking at a two-tailed CI, we need to examines values at least as extreme as the values observed at both sides of the interval, which means higher than 0.1 or lower than -0.1; however, it does not make a lot of sense because we can expect that the genie will never bias the data in his disfavor.
The number of times we get a result at least as extreme as our difference of the means is  662 .


In [19]:
### Calculate the number 2-tailed p-value

#A p-value measures the probability of obtaining the observed results, assuming that the null hypothesis is true.

print('The pvalue computed on this distribution is ', extreme/n,'.')
print(extreme/n<alpha)
print('0.1 is not in the CI, and the p value is inferior to alpha: the data makes sense.')

The pvalue computed on this distribution is  0.00662 .
True
0.1 is not in the CI, and the p value is inferior to alpha: the data makes sense.


##### v) Let's recalculate CI and p-value as one-tailed hypothesis tests with alpha = 0.01

In [20]:
### CI: write a function based on the code we have in class that calculates the 1-sided CI
def onetailedCI(stds, n=n, alpha=alpha):
    return [-float('inf'), round(stds[int(n*(1-alpha))], 5)]

In [21]:
### p-value: write a function that calculates the equivalent one-tailed p-value

def pval(stds, n=n):
    extreme=len([i for i in stds if i>=0.1])
    return extreme/n

In [22]:
### try varying alpha and recalculate the CI and p-value, store the values in one pandas dataframe for the one tailed
### and display the dataframe here
import numpy as np
alpha_range=list(np.arange(0.0001, 0.05, 0.0001))
CI_range=[]
p_value_range=[]
for alpha in alpha_range:
    CI_range.append(onetailedCI(stds,n, alpha))
    p_value_range.append(pval(stds, n=n))
df=pd.DataFrame([alpha_range, CI_range, p_value_range], index=['alpha', 'CI', 'p_value'])
df=df.transpose()
df["Coherent data"]=[(0.1 >df['CI'].iloc[i][1] and df['p_value'].iloc[i]<df['alpha'].iloc[i]) or (0.1 <=df['CI'].iloc[i][1] and df['p_value'].iloc[i]>=df['alpha'].iloc[i])  for i in range (len(df))]
df

,alpha,CI,p_value,Coherent data
0,0.0001,"[-inf, 0.12222]",0.00197,True
1,0.0002,"[-inf, 0.11778]",0.00197,True
2,0.0003,"[-inf, 0.11556]",0.00197,True
3,0.0004,"[-inf, 0.11444]",0.00197,True
4,0.0005,"[-inf, 0.11222]",0.00197,True
...,...,...,...,...
494,0.0495,"[-inf, 0.06]",0.00197,True
495,0.0496,"[-inf, 0.06]",0.00197,True
496,0.0497,"[-inf, 0.06]",0.00197,True
497,0.0498,"[-inf, 0.06]",0.00197,True


In [23]:
#with the list of proposed alphas:
alphas=[0.01,0.0021, 0.001, 0.0005, 0.0001, 0.00001]
CI_range=[]
p_value_range=[]
for alpha in alphas:
    CI_range.append(onetailedCI(stds,n, alpha))
    p_value_range.append(pval(stds, n=n))
df2=pd.DataFrame([alphas, CI_range, p_value_range], index=['alpha', 'CI', 'p_value'])
df2=df2.transpose()
df2["Coherent data"]=[(0.1 >df2['CI'].iloc[i][1] and df2['p_value'].iloc[i]<df2['alpha'].iloc[i]) or (0.1 <df2['CI'].iloc[i][1] and df2['p_value'].iloc[i]>df2['alpha'].iloc[i])  for i in range (len(df2))]
df2['Is_likely_cheating with 1-alpha confidence']=[0.1 >df2['CI'].iloc[i][1] for i in range(len(df2))]
df2

,alpha,CI,p_value,Coherent data,Is_likely_cheating with 1-alpha confidence
0,0.01,"[-inf, 0.08333]",0.00197,True,True
1,0.0021,"[-inf, 0.09889]",0.00197,True,True
2,0.001,"[-inf, 0.10556]",0.00197,True,False
3,0.0005,"[-inf, 0.11222]",0.00197,True,False
4,0.0001,"[-inf, 0.12222]",0.00197,True,False
5,1e-05,"[-inf, 0.13444]",0.00197,True,False


##### comment on your results